In [1]:
import pandas as pd
import polars as pl
import geopandas as gpd

In [4]:
filtered_parquet = "/home/jupyter/datasphere/project/filtered_205.parquet"
df1 = pl.read_parquet(filtered_parquet).to_pandas()

In [5]:
df2 = pl.read_parquet("from-s3-folder/fixed_part-0.parquet").to_pandas()

In [6]:
df1.tin = df1.tin.apply(lambda el: int(el))

In [7]:
df2 = df2.dropna(subset=['inn'])
df2.inn = df2.inn.apply(lambda el: int(el))

In [8]:
df3 = pd.merge(df1, df2, left_on='tin', right_on='inn', how='inner')

In [9]:
#df3 = df3.dropna(subset=['revenue'])
df3 = df3[df3['year'] == 2023]

In [10]:
df3 = df3[df3['activity_code_main'].isin([
    "56.10", "56.10.1", "56.10.2", "56.10.3", "56.30"])]

In [75]:
df3[df3['org_short_name'].str.contains("ЮДЖИН", na=False)][["org_short_name", "lat_y", "lon_y"]]


,org_short_name,lat_y,lon_y
4888782,"ООО ""ЮДЖИН""",55.6857161,37.4776388


In [11]:
df4 = pd.read_csv("coffee_shops_final.csv")

In [12]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1034 entries, 0 to 1033
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1034 non-null   int64  
 1   name              1034 non-null   object 
 2   lat               1034 non-null   float64
 3   lng               1034 non-null   float64
 4   Mon_start         1005 non-null   object 
 5   Mon_end           1005 non-null   object 
 6   Tue_start         1013 non-null   object 
 7   Tue_end           1013 non-null   object 
 8   Wed_start         1013 non-null   object 
 9   Wed_end           1013 non-null   object 
 10  Thu_start         1013 non-null   object 
 11  Thu_end           1013 non-null   object 
 12  Fri_start         1012 non-null   object 
 13  Fri_end           1012 non-null   object 
 14  Sat_start         975 non-null    object 
 15  Sat_end           975 non-null    object 
 16  Sun_start         965 non-null    object 


In [13]:
df4_geom = gpd.points_from_xy(df4.lat, df4.lng)
df4_geo = gpd.GeoDataFrame(df4, geometry=df4_geom, crs="EPSG:4326")

In [14]:
df3_geom = gpd.points_from_xy(df3.lat_y, df3.lon_y)
df3_geo = gpd.GeoDataFrame(df3, geometry=df3_geom, crs="EPSG:4326")

In [15]:
df3_geo = df3_geo.reset_index()
df4_geo = df4_geo.reset_index()

In [16]:
d1t = df3_geo[df3_geo['org_short_name'].str.contains("ЮДЖИН", na=False)]
d2t = df4_geo.head()

In [17]:
d1t

,index,tin,year,reg_number,kind,category,org_name,org_short_name,activity_code_main,region_iso_code,region_code,region_x,area,settlement,settlement_type,oktmo_x,lat_x,lon_x,start_date,end_date,revenue,expenditure,employees_count,inn,ogrn,region_y,region_taxcode,creation_date,dissolution_date,age,eligible,exemption_criteria,financial,filed,imputed,simplified,articulated,totals_adjustment,outlier,okved,...,line_4311,line_4312,line_4313,line_4314,line_4319,line_431x,line_4320,line_4321,line_4322,line_4323,line_4329,line_432x,line_4400,line_4450,line_4490,line_4500,line_6100,line_6200,line_6210,line_6215,line_6220,line_6230,line_6240,line_6250,line_6300,line_6310,line_6311,line_6312,line_6313,line_6320,line_6321,line_6322,line_6323,line_6324,line_6325,line_6326,line_6330,line_6350,line_6400,geometry
16235,4888782,9729021580,2023,1.167747e+12,1,1,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЮДЖИН""","ООО ""ЮДЖИН""",56.10,RU-MOW,77,Москва,None,Москва,г,45000000,55.754047,37.620403,2016-09-10,2024-06-10,78910000.0,72838000.0,15.0,9729021580,1167746782708,moscow city,77,2016-08-19,NaT,7.0,1.0,None,0.0,1.0,0.0,1.0,1.0,1.0,0.0,56.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (55.68572 37.47764)


In [18]:
d1t = d1t.loc[np.repeat(d1t.index.values, 5)]

NameError: name 'np' is not defined

In [174]:
d1t.geometry

16235    POINT (55.68572 37.47764)
16235    POINT (55.68572 37.47764)
16235    POINT (55.68572 37.47764)
16235    POINT (55.68572 37.47764)
16235    POINT (55.68572 37.47764)
Name: geometry, dtype: geometry

In [152]:
d2t.geometry

0    POINT (55.70351 37.51115)
1    POINT (55.67939 37.46827)
2    POINT (55.75444 37.52397)
3    POINT (55.67144 37.57684)
4    POINT (55.63679 37.72424)
Name: geometry, dtype: geometry

In [170]:
d1t.to_crs("EPSG:28416").reset_index().geometry

0    POINT (13150994.128 4871597.972)
1    POINT (13150994.128 4871597.972)
2    POINT (13150994.128 4871597.972)
3    POINT (13150994.128 4871597.972)
4    POINT (13150994.128 4871597.972)
Name: geometry, dtype: geometry

In [171]:
d2t.to_crs("EPSG:28416").geometry

0      POINT (13154409.53 4874691.03)
1    POINT (13149915.341 4870790.919)
2    POINT (13159751.667 4874001.624)
3    POINT (13154985.189 4883595.044)
4    POINT (13159715.949 4901979.285)
Name: geometry, dtype: geometry

In [ ]:
d3t

In [49]:
df4_geo.columns

Index(['index', 'Unnamed: 0', 'name', 'lat', 'lng', 'Mon_start', 'Mon_end',
       'Tue_start', 'Tue_end', 'Wed_start', 'Wed_end', 'Thu_start', 'Thu_end',
       'Fri_start', 'Fri_end', 'Sat_start', 'Sat_end', 'Sun_start', 'Sun_end',
       'espresso_price', 'cappuchino_price', 'specialty_cacao', 'v60',
       'sells_beans', 'dog_friendly', 'with_kitchen', 'bakery',
       'laptop_friendly', 'decaf', 'geometry'],
      dtype='object')

In [50]:
df4_geo.drop(columns=["index", "Unnamed: 0"])

,name,lat,lng,Mon_start,Mon_end,Tue_start,Tue_end,Wed_start,Wed_end,Thu_start,Thu_end,Fri_start,Fri_end,Sat_start,Sat_end,Sun_start,Sun_end,espresso_price,cappuchino_price,specialty_cacao,v60,sells_beans,dog_friendly,with_kitchen,bakery,laptop_friendly,decaf,geometry
0,milk-beans,55.703506,37.511152,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,150,240,False,True,True,True,False,False,False,False,POINT (55.70351 37.51115)
1,milk-beans,55.679390,37.468272,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,150,240,False,True,True,True,False,False,False,False,POINT (55.67939 37.46827)
2,milk-beans,55.754441,37.523975,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,150,240,False,True,True,True,False,False,False,False,POINT (55.75444 37.52397)
3,milk-beans,55.671443,37.576835,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,150,240,False,True,True,True,False,False,False,False,POINT (55.67144 37.57684)
4,milk-beans,55.636788,37.724239,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,10:00,22:00,10:00,22:00,150,240,False,True,True,True,False,False,False,False,POINT (55.63679 37.72424)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,shhegol,59.935057,30.333794,10:00,22:00,10:00,22:00,10:00,22:00,10:00,22:00,10:00,22:00,10:00,22:00,10:00,22:00,110,180,False,True,True,True,False,False,False,False,POINT (59.93506 30.33379)
1030,eklernaya-kler,55.770496,37.633162,9:00,21:00,9:00,21:00,9:00,21:00,9:00,21:00,9:00,21:00,9:00,21:00,9:00,21:00,100,150,False,False,False,True,False,False,False,False,POINT (55.7705 37.63316)
1031,eklernaya-kler,55.772491,37.675998,8:00,21:00,8:00,21:00,8:00,21:00,8:00,21:00,8:00,21:00,8:00,21:00,8:00,21:00,100,150,False,False,False,True,False,False,False,False,POINT (55.77249 37.676)
1032,erna,55.732911,37.618316,8:00,19:30,8:00,21:00,8:00,21:00,8:00,21:00,8:00,21:00,9:30,21:00,9:30,21:00,130,200,False,True,True,True,False,False,False,False,POINT (55.73291 37.61832)


In [173]:
d1t.to_crs("EPSG:28416").reset_index().distance(d2t.to_crs("EPSG:28416"))

0     4607.817724
1     1347.261604
2     9081.411738
3    12643.508705
4    31608.453570
dtype: float64

In [46]:
new_df = df4_geo.to_crs("EPSG:3857") \
    .sjoin_nearest(df3_geo.to_crs("EPSG:3857"), distance_col="dist") \
    [["name", "org_short_name", "dist", "lat_y", "lon_y", "revenue", "expenditure", "geometry"]] \
    .dropna(subset=["revenue"])
new_df = new_df[new_df["revenue"] != 0.0]
new_df.to_csv("second_merge_test.csv")

In [ ]:
new_df)2

In [35]:
dfmsk = pd.read_csv("msc-coffe.csv")
dfspb = pd.read_csv("spb-coffe.csv")

In [39]:
df_zoon = pd.concat([dfmsk, dfspb])

In [43]:
df_zoon_geom = gpd.points_from_xy(df_zoon.lat, df_zoon.lon)
df_zoon_geo = gpd.GeoDataFrame(df_zoon, geometry=df_zoon_geom, crs="EPSG:4326")

In [44]:
df_zoon_geo

,city,name,zoon_id,rating,reviews_count,price_category,tags,address,metro_station,work_hours_short,phone,lat,lon,photo_count,url,photos_urls,geometry
0,msk,Школьник,5a0b5433a24fd95ca1560c1c,2.9,3.0,1/4,кофейня,"улица Земляной Вал, 12/7 ст1, 1 этаж, Москва",Курская,Открыто до 22:00,+7 (985) 984-09-25,55.761731,37.656844,50.0,https://zoon.ru/msk/restaurants/kafe_shkolnik_...,https://pr6.zoon.ru/LVJ-i_NsI_Xle6tWJPVpNA/252...,POINT (55.76173 37.65684)
1,msk,Coffeemolka,58f9bed9504e8d2c7b6a358f,4.2,5.0,1/4,кофейня,"улица Зеленодольская, 45 корпус 1, Москва",Кузьминки,Открыто до 20:00,+7 (909) 973-27-29,55.701928,37.765984,2.0,https://zoon.ru/msk/restaurants/kofejnya_coffe...,https://p0.zoon.ru/preview/j36vmLmnzU1lMyy-YSO...,POINT (55.70193 37.76598)
2,msk,Шоколадница,4f85bd553c72dd8114001c66,3.0,2.0,1/4,кофейня,"Международный Аэропорт Шереметьево, терминал B...",NaN,Круглосуточно,+7 (906) 048-47-34,55.980913,37.410896,51.0,https://zoon.ru/msk/restaurants/kofejnya_shoko...,https://pr6.zoon.ru/yhQX_OLRTGECBRGsHQd_Jg/252...,POINT (55.98091 37.4109)
3,msk,Кофе Хауз,4f85bd513c72dd8114001238,3.7,2.0,1/4,"кондитерская, кофейня","Ильинка, 13/19, стр. 2, Москва",Китай-город,Круглосуточно,+7 (495) 221-83-81,55.755937,37.628088,55.0,https://zoon.ru/msk/restaurants/kafe_kofe_hauz...,https://pr6.zoon.ru/wdKQAXyyiZhgS6bVBcifiQ/252...,POINT (55.75594 37.62809)
4,msk,Шоколадница,4f85bd513c72dd81140012b4,2.9,3.0,1/4,кофейня,"Дмитровское шоссе, 89, в ТЦ ""XL"", 1 этаж, Москва",Селигерская,Открыто до 22:00,+7 (985) 310-40-32,55.863789,37.544854,51.0,https://zoon.ru/msk/restaurants/kofejnya_shoko...,https://pr6.zoon.ru/A6ccfQsUdIMHfYVzJh8VdQ/252...,POINT (55.86379 37.54485)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,spb,Цех85,5a0e6b64a24fd967b92e4c75,4.8,1.0,1/4,"кондитерская, кофейня, пекарня, суши-бар","Массив Скотное, 1, 1 этаж, Санкт-Петербург",NaN,Открыто до 22:00,+7 (800) 500-89-85,60.163256,30.355865,39.0,https://zoon.ru/spb/restaurants/pekarnya-kondi...,https://pr5.zoon.ru/CFehwzp4kIrvoQpl-3sOEw/252...,POINT (60.16326 30.35586)
583,spb,Цех85,57ce700640c0886a548ba260,4.6,1.0,1/4,"кондитерская, кофейня, пекарня, суши-бар","улица Ленсовета, 70, Санкт-Петербург",Звёздная,Открыто до 22:00,+7 (800) 500-89-85,59.835650,30.344663,71.0,https://zoon.ru/spb/restaurants/pekarnya-kondi...,https://pr5.zoon.ru/E36cfVAzctqa7LIBTLpLiQ/252...,POINT (59.83565 30.34466)
584,spb,Hochu coffee,6164a74cb223d44818030322,4.9,4.0,1/4,кофейня,"набережная Обводного канала, 122, Санкт-Петербург",Балтийская,Открыто до 22:00,+7 (904) 640-71-66,59.907857,30.297613,8.0,https://zoon.ru/spb/restaurants/kofejnya_hochu...,https://p2.zoon.ru/preview/6Nt_QBIiK51HoXzy4sQ...,POINT (59.90786 30.29761)
585,spb,"Знакомьтесь, Джо!",52b02a4e40c0888d378b9557,4.4,70.0,1/4,"бар, кофейня","Литейный проспект, 57, 1 этаж; от входа направ...",Достоевская,Открыто до 22:00,+7 (981) 149-92-19,59.934644,30.347749,3.0,https://zoon.ru/spb/restaurants/kofejnya_znako...,https://p0.zoon.ru/preview/_k9tXpMtfp-Rf6SuOSo...,POINT (59.93464 30.34775)


In [55]:
new_df.sjoin_nearest(df_zoon_geo.to_crs("EPSG:3857"), distance_col="dist2").to_csv("second_merge_test.csv")

In [160]:
import numpy as np
from scipy.spatial import KDTree
import geopandas as gpd
import warnings

def find_k_closest(gdf1, gdf2, k):
    # Clean input data
    gdf1_clean = validate_geometries(gdf1)
    gdf2_clean = validate_geometries(gdf2)
    
    # Adjust k for available points
    k = min(k, len(gdf2_clean))
    if k <= 0:
        return gpd.pd.DataFrame(columns=['origin_index', 'neighbor_index', 'distance'])
    
    # Extract finite coordinates
    coords1 = extract_coords(gdf1_clean)
    coords2 = extract_coords(gdf2_clean)
    
    # Build and query KDTree
    tree = KDTree(coords2)
    distances, indices = tree.query(coords1, k=k)
    
    # Handle k=1 edge case
    if k == 1:
        distances = distances[:, None]
        indices = indices[:, None]
    
    # Generate results
    results = []
    for i, orig_idx in enumerate(gdf1_clean.index):
        for j in range(k):
            neighbor_idx = gdf2_clean.index[indices[i, j]]
            results.append({
                'origin_index': orig_idx,
                'neighbor_index': neighbor_idx,
                'distance': distances[i, j]
            })
    
    return gpd.pd.DataFrame(results)

# Helper functions
def validate_geometries(gdf):
    valid_mask = (
        gdf.geometry.notna()
        & (gdf.geometry.type == 'Point')
        & gdf.geometry.apply(lambda g: all(np.isfinite([g.x, g.y])))
    )
    if not valid_mask.all():
        invalid = gdf[~valid_mask]
        warnings.warn(
            f"Dropped {len(invalid)} invalid geometries (indexes: {invalid.index.tolist()})"
        )
    return gdf[valid_mask].copy()

def extract_coords(gdf):
    return np.column_stack([gdf.geometry.x, gdf.geometry.y])

In [175]:
find_k_closest(df4_geo.to_crs("EPSG:28416")[["name", "geometry"]].head(), df3_geo.to_crs("EPSG:28416").dropna(subset=["geometry"])[["org_short_name", "geometry", "revenue"]], 10)

/tmp/ipykernel_4547/1877443122.py:51: UserWarning: Dropped 3 invalid geometries (indexes: [1470, 7624, 9133])
  warnings.warn(


,origin_index,neighbor_index,distance
0,0,16438,15.534636
1,0,1398,161.376424
2,0,16397,434.752994
3,0,16449,523.180785
4,0,16330,740.915162
5,0,5253,740.915162
6,0,16372,740.915162
7,0,16373,740.915162
8,0,16270,740.915162
9,0,4422,740.915162
